In [1]:
from sklearn.preprocessing import OrdinalEncoder, PolynomialFeatures
from sklearn.model_selection import GridSearchCV
from pycaret.regression import setup, compare_models
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

In [2]:
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, ExtraTreesRegressor
from sklearn.linear_model import BayesianRidge, OrthogonalMatchingPursuit, LinearRegression, Lasso, Ridge
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

# Read train and test sets

In [3]:
df_train = pd.read_csv("data/train.csv")
df_test = pd.read_csv("data/test.csv")

# Store both dfs lengths

In [4]:
m_train = df_train.shape[0]
m_test = df_test.shape[0]
m_train, m_test

(1460, 1459)

# Concat the dfs

In [5]:
df = pd.concat([df_train, df_test])
assert df.shape[0] == m_train + m_test

In [6]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500.0
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500.0
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500.0
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000.0
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000.0


# Save target

In [7]:
target_col, target = "SalePrice", df["SalePrice"]

#  Drop Id and target columns

In [8]:
cols_to_drop = ["Id", target_col]
df.drop(cols_to_drop, axis=1, inplace=True)

# View and fix columns data-types

In [9]:
df.select_dtypes(object).columns

Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition'],
      dtype='object')

In [10]:
df.select_dtypes(np.number).columns

Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
       'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
       'MoSold', 'YrSold'],
      dtype='object')

In [11]:
num_to_obj_cols = ['MSSubClass', 'MoSold']
df[num_to_obj_cols] = df[num_to_obj_cols].astype(object)

In [12]:
cols_cat = df.select_dtypes(object).columns.to_list()
cols_num = df.select_dtypes(np.number).columns.to_list()

# Impute categorical columns

In [13]:
cols_cat_na = df[cols_cat].isnull().sum()[df[cols_cat].isnull().sum() > 0]
cols_cat_na

MSZoning           4
Alley           2721
Utilities          2
Exterior1st        1
Exterior2nd        1
MasVnrType        24
BsmtQual          81
BsmtCond          82
BsmtExposure      82
BsmtFinType1      79
BsmtFinType2      80
Electrical         1
KitchenQual        1
Functional         2
FireplaceQu     1420
GarageType       157
GarageFinish     159
GarageQual       159
GarageCond       159
PoolQC          2909
Fence           2348
MiscFeature     2814
SaleType           1
dtype: int64

In [14]:
mode_filled_cols = ["MSZoning", "Utilities", "Exterior1st", "Exterior2nd", "MasVnrType", "Electrical", "KitchenQual", "Functional", "SaleType"]
for col in mode_filled_cols:
    df[col].fillna(df[col].mode()[0], inplace=True)

none_filled_cols = ["Alley", "BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1", "BsmtFinType2", "FireplaceQu", "GarageType", "GarageFinish", "GarageQual", "GarageCond", "PoolQC", "Fence", "MiscFeature"]
for col in none_filled_cols:
    df[col].fillna("None", inplace=True)
    
df[cols_cat].isnull().sum().sum()

0

# Change ordinal columns to numeric, and encode accordingly

In [15]:
df.select_dtypes(object).columns

Index(['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour',
       'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1',
       'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl',
       'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond',
       'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
       'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical',
       'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType',
       'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC',
       'Fence', 'MiscFeature', 'MoSold', 'SaleType', 'SaleCondition'],
      dtype='object')

In [16]:
oe = OrdinalEncoder(categories=[['Reg', 'IR1', 'IR2', 'IR3']])
df.loc[:, "LotShape"] = oe.fit_transform(df[["LotShape"]])
# df["LotShape_sq"] = df["LotShape"] ** 2

In [17]:
oe = OrdinalEncoder(categories=[['Gtl', 'Mod', 'Sev']])
df.loc[:, "LandSlope"] = oe.fit_transform(df[["LandSlope"]])
# df["LandSlope_sq"] = df["LandSlope"] ** 2

In [18]:
qual_oe = OrdinalEncoder(categories=[['None', 'Po', 'Fa', 'TA', 'Gd', 'Ex']])
for col in ["ExterQual", "ExterCond", 'BsmtQual', 'BsmtCond', 'HeatingQC', 'KitchenQual', 'FireplaceQu', 'GarageQual', 'GarageCond', 'PoolQC']:
    df.loc[:, col] = qual_oe.fit_transform(df[[col]])
#     df[col + "sq"] = df[col] ** 2

In [19]:
oe = OrdinalEncoder(categories=[['None', 'No', 'Mn', 'Av', 'Gd']])
df.loc[:, "BsmtExposure"] = oe.fit_transform(df[["BsmtExposure"]])
# df["BsmtExposure_sq"] = df["BsmtExposure"] ** 2

In [20]:
oe = OrdinalEncoder(categories=[['None', 'Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ']])
for col in ["BsmtFinType1", "BsmtFinType2"]:
    df.loc[:, col] = oe.fit_transform(df[[col]])
#     df[col + "sq"] = df[col] ** 2

In [21]:
oe = OrdinalEncoder(categories=[['None', 'Unf', 'RFn', 'Fin']])
df.loc[:, "GarageFinish"] = oe.fit_transform(df[["GarageFinish"]])
# df["GarageFinishsq"] = df["GarageFinish"] ** 2

In [22]:
oe = OrdinalEncoder(categories=[['N', 'P', 'Y']])
df.loc[:, "PavedDrive"] = oe.fit_transform(df[["PavedDrive"]])
# df["PavedDrivesq"] = df["PavedDrive"] ** 2

In [23]:
oe = OrdinalEncoder(categories=[['None', 'MnWw', 'GdWo', 'MnPrv', 'GdPrv']])
df.loc[:, "Fence"] = oe.fit_transform(df[["Fence"]])
# df["Fencesq"] = df["Fence"] ** 2

In [24]:
cols_cat = df.select_dtypes(object).columns.to_list()
cols_num = df.select_dtypes(np.number).columns.to_list()

# Impute numerical columns

In [25]:
cols_num_na = df[cols_num].isnull().sum()[df[cols_num].isnull().sum() > 0]
cols_num_na

LotFrontage     486
MasVnrArea       23
BsmtFinSF1        1
BsmtFinSF2        1
BsmtUnfSF         1
TotalBsmtSF       1
BsmtFullBath      2
BsmtHalfBath      2
GarageYrBlt     159
GarageCars        1
GarageArea        1
dtype: int64

In [26]:
zero_filled_cols = ["BsmtFinSF1", "BsmtFinSF2", "BsmtUnfSF", "TotalBsmtSF", "BsmtFullBath", "BsmtHalfBath", "GarageCars", "GarageArea"]
for col in zero_filled_cols:
    df[col].fillna(0, inplace=True)

In [27]:
cols_num_na = df[cols_num].isnull().sum()[df[cols_num].isnull().sum() > 0]
cols_num_na

LotFrontage    486
MasVnrArea      23
GarageYrBlt    159
dtype: int64

In [28]:
def catboost_imputer(df, cols_num_na):
    
    # save columns to impute and drop them from df
    cols_to_impute = df[cols_num_na]
    df.drop(cols_num_na, axis=1, inplace=True)
    
    for col in cols_num_na:
        # define X and y
        X, y = df.copy(), cols_to_impute[col]

        # get train and test sets
        train_indexes, test_indexes = ~y.isnull(), y.isnull()
        X_train, X_test = X.loc[train_indexes, :], X.loc[test_indexes, :]
        y_train, y_test = y.loc[train_indexes], y.loc[test_indexes]
        
        model = CatBoostRegressor(max_depth=8, random_seed=10,
                                  subsample=0.65, n_estimators=1000,
                                  cat_features=cols_cat)
        model.fit(X_train, y_train)
        cols_to_impute.loc[test_indexes, col] = model.predict(X_test)
        df[col] = cols_to_impute[col]
    return df

In [29]:
df = catboost_imputer(df, cols_num_na.index)
df.isnull().sum().sum()

Learning rate set to 0.047118
0:	learn: 22.8881607	total: 87ms	remaining: 1m 26s
1:	learn: 22.4167077	total: 102ms	remaining: 50.7s
2:	learn: 21.9674201	total: 117ms	remaining: 38.8s
3:	learn: 21.5550892	total: 127ms	remaining: 31.6s
4:	learn: 21.1728677	total: 140ms	remaining: 27.9s
5:	learn: 20.8205778	total: 151ms	remaining: 25.1s
6:	learn: 20.4210801	total: 167ms	remaining: 23.6s
7:	learn: 20.0631293	total: 181ms	remaining: 22.4s
8:	learn: 19.7546986	total: 192ms	remaining: 21.2s
9:	learn: 19.4290106	total: 211ms	remaining: 20.8s
10:	learn: 19.1155800	total: 222ms	remaining: 19.9s
11:	learn: 18.8276333	total: 238ms	remaining: 19.6s
12:	learn: 18.5296271	total: 252ms	remaining: 19.1s
13:	learn: 18.2768483	total: 267ms	remaining: 18.8s
14:	learn: 18.0281784	total: 279ms	remaining: 18.3s
15:	learn: 17.7467394	total: 296ms	remaining: 18.2s
16:	learn: 17.4911819	total: 309ms	remaining: 17.9s
17:	learn: 17.2760921	total: 323ms	remaining: 17.6s
18:	learn: 17.0819915	total: 337ms	remaining

161:	learn: 10.9779874	total: 2.29s	remaining: 11.9s
162:	learn: 10.9757887	total: 2.31s	remaining: 11.9s
163:	learn: 10.9705998	total: 2.33s	remaining: 11.9s
164:	learn: 10.9630915	total: 2.34s	remaining: 11.8s
165:	learn: 10.9283631	total: 2.36s	remaining: 11.8s
166:	learn: 10.9214767	total: 2.37s	remaining: 11.8s
167:	learn: 10.9062928	total: 2.38s	remaining: 11.8s
168:	learn: 10.8996323	total: 2.4s	remaining: 11.8s
169:	learn: 10.8759552	total: 2.41s	remaining: 11.8s
170:	learn: 10.8610464	total: 2.42s	remaining: 11.8s
171:	learn: 10.8501047	total: 2.44s	remaining: 11.8s
172:	learn: 10.8206732	total: 2.45s	remaining: 11.7s
173:	learn: 10.7984382	total: 2.47s	remaining: 11.7s
174:	learn: 10.7626695	total: 2.48s	remaining: 11.7s
175:	learn: 10.7626230	total: 2.49s	remaining: 11.6s
176:	learn: 10.7562282	total: 2.5s	remaining: 11.6s
177:	learn: 10.7410752	total: 2.51s	remaining: 11.6s
178:	learn: 10.6987336	total: 2.53s	remaining: 11.6s
179:	learn: 10.6868306	total: 2.54s	remaining: 1

330:	learn: 8.8073084	total: 4.8s	remaining: 9.7s
331:	learn: 8.7998563	total: 4.82s	remaining: 9.69s
332:	learn: 8.7874218	total: 4.83s	remaining: 9.68s
333:	learn: 8.7758663	total: 4.85s	remaining: 9.66s
334:	learn: 8.7667110	total: 4.86s	remaining: 9.65s
335:	learn: 8.7518400	total: 4.87s	remaining: 9.63s
336:	learn: 8.7429349	total: 4.89s	remaining: 9.62s
337:	learn: 8.7320831	total: 4.9s	remaining: 9.6s
338:	learn: 8.7310364	total: 4.92s	remaining: 9.59s
339:	learn: 8.7250007	total: 4.93s	remaining: 9.57s
340:	learn: 8.7193640	total: 4.95s	remaining: 9.56s
341:	learn: 8.7188219	total: 4.96s	remaining: 9.55s
342:	learn: 8.7101290	total: 4.98s	remaining: 9.53s
343:	learn: 8.6968252	total: 4.99s	remaining: 9.52s
344:	learn: 8.6754105	total: 5.01s	remaining: 9.51s
345:	learn: 8.6633886	total: 5.02s	remaining: 9.49s
346:	learn: 8.6516296	total: 5.04s	remaining: 9.47s
347:	learn: 8.6362039	total: 5.05s	remaining: 9.47s
348:	learn: 8.6244988	total: 5.07s	remaining: 9.45s
349:	learn: 8.61

500:	learn: 7.4855485	total: 7.31s	remaining: 7.28s
501:	learn: 7.4701740	total: 7.33s	remaining: 7.27s
502:	learn: 7.4622350	total: 7.34s	remaining: 7.25s
503:	learn: 7.4530071	total: 7.36s	remaining: 7.24s
504:	learn: 7.4490366	total: 7.37s	remaining: 7.22s
505:	learn: 7.4445984	total: 7.39s	remaining: 7.21s
506:	learn: 7.4439574	total: 7.4s	remaining: 7.2s
507:	learn: 7.4420778	total: 7.42s	remaining: 7.18s
508:	learn: 7.4417615	total: 7.43s	remaining: 7.17s
509:	learn: 7.4317259	total: 7.45s	remaining: 7.16s
510:	learn: 7.4204709	total: 7.46s	remaining: 7.14s
511:	learn: 7.4094249	total: 7.47s	remaining: 7.12s
512:	learn: 7.4073005	total: 7.49s	remaining: 7.11s
513:	learn: 7.3941922	total: 7.5s	remaining: 7.09s
514:	learn: 7.3929468	total: 7.52s	remaining: 7.08s
515:	learn: 7.3736044	total: 7.53s	remaining: 7.06s
516:	learn: 7.3591427	total: 7.55s	remaining: 7.05s
517:	learn: 7.3538325	total: 7.56s	remaining: 7.03s
518:	learn: 7.3494038	total: 7.57s	remaining: 7.02s
519:	learn: 7.3

667:	learn: 6.4874702	total: 9.8s	remaining: 4.87s
668:	learn: 6.4839869	total: 9.81s	remaining: 4.86s
669:	learn: 6.4836687	total: 9.83s	remaining: 4.84s
670:	learn: 6.4812258	total: 9.84s	remaining: 4.83s
671:	learn: 6.4800726	total: 9.86s	remaining: 4.81s
672:	learn: 6.4797120	total: 9.87s	remaining: 4.8s
673:	learn: 6.4771672	total: 9.89s	remaining: 4.78s
674:	learn: 6.4686327	total: 9.9s	remaining: 4.77s
675:	learn: 6.4582340	total: 9.92s	remaining: 4.75s
676:	learn: 6.4571020	total: 9.93s	remaining: 4.74s
677:	learn: 6.4513959	total: 9.95s	remaining: 4.72s
678:	learn: 6.4370494	total: 9.96s	remaining: 4.71s
679:	learn: 6.4281776	total: 9.97s	remaining: 4.69s
680:	learn: 6.4250043	total: 9.99s	remaining: 4.68s
681:	learn: 6.4115063	total: 10s	remaining: 4.66s
682:	learn: 6.4111358	total: 10s	remaining: 4.65s
683:	learn: 6.4109314	total: 10s	remaining: 4.64s
684:	learn: 6.4104357	total: 10.1s	remaining: 4.62s
685:	learn: 6.4033522	total: 10.1s	remaining: 4.61s
686:	learn: 6.3980376

...

492:	learn: 5.5512957	total: 7.54s	remaining: 7.76s
493:	learn: 5.5509428	total: 7.57s	remaining: 7.75s
494:	learn: 5.5463145	total: 7.58s	remaining: 7.73s
495:	learn: 5.5451724	total: 7.6s	remaining: 7.72s
496:	learn: 5.5396927	total: 7.61s	remaining: 7.71s
497:	learn: 5.5362120	total: 7.63s	remaining: 7.69s
498:	learn: 5.5360028	total: 7.65s	remaining: 7.68s
499:	learn: 5.5328047	total: 7.67s	remaining: 7.67s
500:	learn: 5.5305495	total: 7.68s	remaining: 7.65s
501:	learn: 5.5264455	total: 7.7s	remaining: 7.64s
502:	learn: 5.5220233	total: 7.71s	remaining: 7.62s
503:	learn: 5.5128830	total: 7.73s	remaining: 7.61s
504:	learn: 5.5108155	total: 7.75s	remaining: 7.59s
505:	learn: 5.5091136	total: 7.77s	remaining: 7.58s
506:	learn: 5.5064488	total: 7.78s	remaining: 7.57s
507:	learn: 5.5012385	total: 7.8s	remaining: 7.56s
508:	learn: 5.5004811	total: 7.82s	remaining: 7.54s
509:	learn: 5.4987492	total: 7.84s	remaining: 7.53s
510:	learn: 5.4865968	total: 7.85s	remaining: 7.51s
511:	learn: 5.4

664:	learn: 4.7095405	total: 10.2s	remaining: 5.16s
665:	learn: 4.7072309	total: 10.3s	remaining: 5.14s
666:	learn: 4.6969541	total: 10.3s	remaining: 5.13s
667:	learn: 4.6845602	total: 10.3s	remaining: 5.11s
668:	learn: 4.6800137	total: 10.3s	remaining: 5.1s
669:	learn: 4.6765023	total: 10.3s	remaining: 5.08s
670:	learn: 4.6748305	total: 10.3s	remaining: 5.07s
671:	learn: 4.6688957	total: 10.3s	remaining: 5.05s
672:	learn: 4.6628641	total: 10.4s	remaining: 5.03s
673:	learn: 4.6628173	total: 10.4s	remaining: 5.02s
674:	learn: 4.6585171	total: 10.4s	remaining: 5s
675:	learn: 4.6498604	total: 10.4s	remaining: 4.99s
676:	learn: 4.6493091	total: 10.4s	remaining: 4.97s
677:	learn: 4.6429960	total: 10.4s	remaining: 4.96s
678:	learn: 4.6420056	total: 10.5s	remaining: 4.94s
679:	learn: 4.6405819	total: 10.5s	remaining: 4.93s
680:	learn: 4.6360435	total: 10.5s	remaining: 4.91s
681:	learn: 4.6267063	total: 10.5s	remaining: 4.9s
682:	learn: 4.6221512	total: 10.5s	remaining: 4.88s
683:	learn: 4.609

824:	learn: 4.0197212	total: 12.7s	remaining: 2.7s
825:	learn: 4.0158341	total: 12.7s	remaining: 2.68s
826:	learn: 4.0096084	total: 12.8s	remaining: 2.67s
827:	learn: 4.0082962	total: 12.8s	remaining: 2.65s
828:	learn: 4.0041910	total: 12.8s	remaining: 2.64s
829:	learn: 3.9992980	total: 12.8s	remaining: 2.62s
830:	learn: 3.9925969	total: 12.8s	remaining: 2.61s
831:	learn: 3.9867050	total: 12.8s	remaining: 2.59s
832:	learn: 3.9861696	total: 12.9s	remaining: 2.58s
833:	learn: 3.9847816	total: 12.9s	remaining: 2.56s
834:	learn: 3.9847252	total: 12.9s	remaining: 2.55s
835:	learn: 3.9762716	total: 12.9s	remaining: 2.53s
836:	learn: 3.9695439	total: 12.9s	remaining: 2.52s
837:	learn: 3.9691929	total: 12.9s	remaining: 2.5s
838:	learn: 3.9682932	total: 13s	remaining: 2.49s
839:	learn: 3.9674168	total: 13s	remaining: 2.47s
840:	learn: 3.9626533	total: 13s	remaining: 2.46s
841:	learn: 3.9623529	total: 13s	remaining: 2.44s
842:	learn: 3.9559673	total: 13s	remaining: 2.42s
843:	learn: 3.9557782	to

995:	learn: 3.4158995	total: 15.4s	remaining: 62ms
996:	learn: 3.4155834	total: 15.5s	remaining: 46.5ms
997:	learn: 3.4128463	total: 15.5s	remaining: 31ms
998:	learn: 3.4081629	total: 15.5s	remaining: 15.5ms
999:	learn: 3.4056666	total: 15.5s	remaining: 0us


0

# Feature selection

In [30]:
def get_del_cols(df):
    # return the columns where no more than one value in the categorical features
    # exists in the test data - those features can be ignored
    cols_to_drop = []
    for col in df.select_dtypes(object).columns:
        col_vals = df[col].unique()
        n_vals = len(col_vals)
        n_irrelevant = 0
        for val in col_vals:
            if val not in df[col][m_train:].values:
                n_irrelevant += 1
        if n_irrelevant >= n_vals - 1:
            cols_to_drop.append(col)
    return cols_to_drop

In [31]:
cols_to_drop = get_del_cols(df)
df.drop(cols_to_drop, axis=1, inplace=True)
for col_to_drop in cols_to_drop:
    cols_cat.remove(col_to_drop)
    print("Dropped ", col_to_drop)

Dropped  Utilities


<!-- # Change ordinal columns to numeric, and encode accordingly -->

# Feature Engineering

## Features from the Internet - prefex FE0_

In [32]:
df["FE0_HighQualSF"] = df["GrLivArea"]+df["1stFlrSF"] + df["2ndFlrSF"]+0.5*df["GarageArea"]+0.5*df["TotalBsmtSF"]+1*df["MasVnrArea"]

df["FE0_SqFtPerRoom"] = df["GrLivArea"] / (df["TotRmsAbvGrd"] +
                                           df["FullBath"] +
                                           df["HalfBath"] +
                                           df["KitchenAbvGr"])

## My features - prefex FE1_

In [33]:
n_stories_dict = {
    "1Story": 1.0,
    "1.5Fin": 1.5,
    "1.5Unf": 1.5,
    "2Story": 2.0,
    "2.5Fin": 2.5,
    "2.5Unf": 2.5,
    "SFoyer": 2.0,
    "SLvl": 2.0,
}

df["FE1_n_stories"] = df["HouseStyle"].replace(n_stories_dict)
df["FE1_n_stories"].value_counts()

1.0    1471
2.0    1083
1.5     333
2.5      32
Name: FE1_n_stories, dtype: int64

In [34]:
df["FE1_age_sold"] = df["YrSold"] - df["YearBuilt"]
df["FE1_age_sold_Remod"] = df["YrSold"] - df["YearRemodAdd"]
df["FE1_GarageYrSold"] = df["YrSold"] - df["GarageYrBlt"]

In [35]:
df["CentralAir"] = df["CentralAir"].replace({"Y": 1, "N": 0})
df["CentralAir"].value_counts()

1    2723
0     196
Name: CentralAir, dtype: int64

In [36]:
df['FE1_total_area_hq'] = df["FE0_HighQualSF"] - df['LowQualFinSF']

In [37]:
df.select_dtypes(np.number).columns

Index(['LotArea', 'LotShape', 'LandSlope', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'ExterQual', 'ExterCond', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'HeatingQC',
       'CentralAir', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea',
       'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr',
       'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Fireplaces',
       'FireplaceQu', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'PoolQC',
       'Fence', 'MiscVal', 'YrSold', 'LotFrontage', 'MasVnrArea',
       'GarageYrBlt', 'FE0_HighQualSF', 'FE0_SqFtPerRoom', 'FE1_n_stories',
       'FE1_age_sold', 'FE1_age_sold_Remod', 'FE1_GarageYrSold',
       'FE1_total_area_hq'],
      dtype='object')

# Identify and apply polynomial features

In [38]:
cols_to_poly = []

cols_to_poly.append(['GarageFinish', 'GarageArea', 'GarageQual', 'GarageCond'])  # garage
cols_to_poly.append(['BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'TotalBsmtSF'])  # bsmt
cols_to_poly.append(['OverallQual', 'OverallCond', 'FE1_total_area_hq'])  # total

In [39]:
for col in df.columns.values:
    if col.startswith("Garage"):
        print(col)

GarageType
GarageFinish
GarageCars
GarageArea
GarageQual
GarageCond
GarageYrBlt


In [40]:
poly = PolynomialFeatures(degree=2, include_bias=False)

for cols in cols_to_poly:
    cols_df = df[cols]
    df.drop(cols, axis=1, inplace=True)
    cols_array = poly.fit_transform(cols_df)
    col_names = ["FE1_" + name for name in poly.get_feature_names_out()]
    cols_df = pd.DataFrame(cols_array, columns=col_names).reset_index(drop=True)
    df = pd.concat((df.reset_index(drop=True), cols_df.reset_index(drop=True)), axis=1)
df.shape

(2919, 129)

In [41]:


# df['FE1_Total_Home_Quality'] = df['OverallQual'] * df['OverallCond']
# df['FE1_Total_bsmt_Quality'] = df['BsmtQual'] * df['BsmtCond']
# df['FE1_Total_garage_Quality'] = df['GarageQual'] * df['GarageCond']

# df["FE1_weighted_home_area"] = df["FE1_total_area_hq"] * df['FE1_Total_Home_Quality'] 
# df["FE1_weighted_bsmt_area"] = df["TotalBsmtSF"] * df['FE1_Total_bsmt_Quality'] 
# df["FE1_weighted_garage_area"] = df["GarageArea"] * df['FE1_Total_garage_Quality']
# df["FE1_weighted_n_stories"] = df["FE1_n_stories"] * df['OverallQual']

# Split test and train

In [42]:
X_train, X_test = df[:m_train], df[m_train:]
y_train, _ = target[:m_train], target[m_train:]
n = X_train.shape[1]
X_train.shape, y_train.shape

((1460, 129), (1460,))

In [43]:
# df_ohe = pd.get_dummies(df)
# df_ohe[target_col] = target
# _ = setup(data=df_ohe[:m_train], target='SalePrice')
# best = compare_models()

# Training - CatBoost Grid Search 

In [44]:
# cat_features = df.select_dtypes(object).columns.to_list()
# model = CatBoostRegressor(random_state=10, cat_features=cat_features)
# kwargs = {
#     "n_estimators": [50 * n],
#     "max_depth": [8],
#     "subsample": [0.65],
#     "reg_lambda": [0.1], 
# }
# clf = GridSearchCV(model, kwargs, verbose=1, n_jobs=2)
# clf.fit(X_train, y_train)
# print(clf.best_score_)
# print(clf.best_params_)

# CatBoost Params:
    {'max_depth': 8, 'n_estimators': 5500, 'reg_lambda': 0.1, 'subsample': 0.65}

In [45]:
# clf.best_score_

# Training - CatBoost with Best Parameters

In [46]:
model = CatBoostRegressor(max_depth=8, random_seed=10,
                          subsample=0.65, n_estimators=11_000,
                          cat_features=cols_cat)
model.fit(X_train, y_train)

Learning rate set to 0.006187
0:	learn: 79091.4416458	total: 15.7ms	remaining: 2m 53s
1:	learn: 78773.6728359	total: 34.6ms	remaining: 3m 10s
2:	learn: 78444.1055520	total: 52.6ms	remaining: 3m 12s
3:	learn: 78142.7663884	total: 70.6ms	remaining: 3m 14s
4:	learn: 77819.1974098	total: 90.4ms	remaining: 3m 18s
5:	learn: 77524.6274418	total: 109ms	remaining: 3m 20s
6:	learn: 77197.8085950	total: 127ms	remaining: 3m 19s
7:	learn: 76865.7558930	total: 144ms	remaining: 3m 17s
8:	learn: 76558.3686221	total: 161ms	remaining: 3m 16s
9:	learn: 76236.5171994	total: 179ms	remaining: 3m 16s
10:	learn: 75946.2953800	total: 196ms	remaining: 3m 15s
11:	learn: 75653.9402604	total: 217ms	remaining: 3m 19s
12:	learn: 75360.1471776	total: 236ms	remaining: 3m 19s
13:	learn: 75046.1477348	total: 254ms	remaining: 3m 19s
14:	learn: 74736.5407282	total: 273ms	remaining: 3m 19s
15:	learn: 74421.3263185	total: 293ms	remaining: 3m 21s
16:	learn: 74124.2630316	total: 314ms	remaining: 3m 22s
17:	learn: 73830.094829

146:	learn: 46732.2890778	total: 2.64s	remaining: 3m 14s
147:	learn: 46592.9119902	total: 2.65s	remaining: 3m 14s
148:	learn: 46462.7083663	total: 2.67s	remaining: 3m 14s
149:	learn: 46325.5476322	total: 2.69s	remaining: 3m 14s
150:	learn: 46193.9190253	total: 2.71s	remaining: 3m 14s
151:	learn: 46044.8511825	total: 2.73s	remaining: 3m 14s
152:	learn: 45891.4027056	total: 2.75s	remaining: 3m 14s
153:	learn: 45762.0267350	total: 2.77s	remaining: 3m 14s
154:	learn: 45634.3114146	total: 2.79s	remaining: 3m 15s
155:	learn: 45484.0440153	total: 2.81s	remaining: 3m 15s
156:	learn: 45342.2684018	total: 2.82s	remaining: 3m 14s
157:	learn: 45207.0366485	total: 2.84s	remaining: 3m 15s
158:	learn: 45073.8593536	total: 2.86s	remaining: 3m 15s
159:	learn: 44939.6980813	total: 2.88s	remaining: 3m 15s
160:	learn: 44804.0646185	total: 2.9s	remaining: 3m 15s
161:	learn: 44676.5412880	total: 2.91s	remaining: 3m 14s
162:	learn: 44553.3111079	total: 2.92s	remaining: 3m 14s
163:	learn: 44424.2420829	total:

297:	learn: 31514.4054142	total: 5.49s	remaining: 3m 17s
298:	learn: 31451.4225879	total: 5.51s	remaining: 3m 17s
299:	learn: 31384.8467817	total: 5.53s	remaining: 3m 17s
300:	learn: 31325.1676246	total: 5.54s	remaining: 3m 17s
301:	learn: 31257.5246053	total: 5.56s	remaining: 3m 17s
302:	learn: 31191.9035573	total: 5.58s	remaining: 3m 17s
303:	learn: 31123.6340053	total: 5.6s	remaining: 3m 17s
304:	learn: 31069.1953325	total: 5.62s	remaining: 3m 17s
305:	learn: 30998.0250899	total: 5.64s	remaining: 3m 17s
306:	learn: 30935.9923666	total: 5.66s	remaining: 3m 17s
307:	learn: 30872.7437339	total: 5.68s	remaining: 3m 17s
308:	learn: 30802.3728394	total: 5.7s	remaining: 3m 17s
309:	learn: 30738.7968372	total: 5.72s	remaining: 3m 17s
310:	learn: 30673.8033169	total: 5.74s	remaining: 3m 17s
311:	learn: 30601.9273060	total: 5.76s	remaining: 3m 17s
312:	learn: 30535.5009369	total: 5.78s	remaining: 3m 17s
313:	learn: 30477.7296166	total: 5.8s	remaining: 3m 17s
314:	learn: 30413.9601084	total: 5

442:	learn: 24661.8649147	total: 8.26s	remaining: 3m 16s
443:	learn: 24636.2200083	total: 8.28s	remaining: 3m 16s
444:	learn: 24604.0885230	total: 8.29s	remaining: 3m 16s
445:	learn: 24576.3075669	total: 8.31s	remaining: 3m 16s
446:	learn: 24546.7008861	total: 8.33s	remaining: 3m 16s
447:	learn: 24516.8787810	total: 8.35s	remaining: 3m 16s
448:	learn: 24483.5322272	total: 8.37s	remaining: 3m 16s
449:	learn: 24449.5499830	total: 8.39s	remaining: 3m 16s
450:	learn: 24416.7499440	total: 8.41s	remaining: 3m 16s
451:	learn: 24385.1908954	total: 8.43s	remaining: 3m 16s
452:	learn: 24354.9801096	total: 8.44s	remaining: 3m 16s
453:	learn: 24324.0966824	total: 8.45s	remaining: 3m 16s
454:	learn: 24291.4482084	total: 8.47s	remaining: 3m 16s
455:	learn: 24260.1058508	total: 8.49s	remaining: 3m 16s
456:	learn: 24229.9670787	total: 8.5s	remaining: 3m 16s
457:	learn: 24202.9897866	total: 8.52s	remaining: 3m 16s
458:	learn: 24168.6642331	total: 8.54s	remaining: 3m 16s
459:	learn: 24139.0198319	total:

596:	learn: 20863.2066397	total: 11.2s	remaining: 3m 15s
597:	learn: 20842.3167384	total: 11.3s	remaining: 3m 15s
598:	learn: 20824.6716886	total: 11.3s	remaining: 3m 15s
599:	learn: 20808.2252367	total: 11.3s	remaining: 3m 15s
600:	learn: 20792.5440327	total: 11.3s	remaining: 3m 15s
601:	learn: 20778.6000891	total: 11.3s	remaining: 3m 15s
602:	learn: 20758.3258370	total: 11.3s	remaining: 3m 15s
603:	learn: 20740.1292974	total: 11.4s	remaining: 3m 15s
604:	learn: 20724.0921513	total: 11.4s	remaining: 3m 15s
605:	learn: 20710.2240823	total: 11.4s	remaining: 3m 15s
606:	learn: 20695.6404696	total: 11.4s	remaining: 3m 15s
607:	learn: 20677.1873463	total: 11.4s	remaining: 3m 15s
608:	learn: 20657.0795501	total: 11.4s	remaining: 3m 15s
609:	learn: 20642.5912060	total: 11.5s	remaining: 3m 15s
610:	learn: 20623.3691817	total: 11.5s	remaining: 3m 15s
611:	learn: 20599.9578891	total: 11.5s	remaining: 3m 15s
612:	learn: 20585.5761444	total: 11.5s	remaining: 3m 15s
613:	learn: 20572.1522568	total

...

10321:	learn: 2744.0780102	total: 3m 33s	remaining: 14.1s
10322:	learn: 2743.9144858	total: 3m 33s	remaining: 14s
10323:	learn: 2743.8983814	total: 3m 33s	remaining: 14s
10324:	learn: 2743.5994432	total: 3m 34s	remaining: 14s
10325:	learn: 2743.5353406	total: 3m 34s	remaining: 14s
10326:	learn: 2743.5314542	total: 3m 34s	remaining: 13.9s
10327:	learn: 2743.2607586	total: 3m 34s	remaining: 13.9s
10328:	learn: 2742.7143223	total: 3m 34s	remaining: 13.9s
10329:	learn: 2742.1996472	total: 3m 34s	remaining: 13.9s
10330:	learn: 2741.6718729	total: 3m 34s	remaining: 13.9s
10331:	learn: 2741.3218497	total: 3m 34s	remaining: 13.8s
10332:	learn: 2740.8644729	total: 3m 34s	remaining: 13.8s
10333:	learn: 2740.3224359	total: 3m 34s	remaining: 13.8s
10334:	learn: 2740.2802190	total: 3m 34s	remaining: 13.8s
10335:	learn: 2739.6604617	total: 3m 34s	remaining: 13.8s
10336:	learn: 2739.1584313	total: 3m 34s	remaining: 13.7s
10337:	learn: 2738.7231929	total: 3m 34s	remaining: 13.7s
10338:	learn: 2738.647

10472:	learn: 2682.3455784	total: 3m 37s	remaining: 10.9s
10473:	learn: 2682.0108679	total: 3m 37s	remaining: 10.9s
10474:	learn: 2681.4569362	total: 3m 37s	remaining: 10.9s
10475:	learn: 2681.1787470	total: 3m 37s	remaining: 10.9s
10476:	learn: 2680.8186351	total: 3m 37s	remaining: 10.8s
10477:	learn: 2680.5605722	total: 3m 37s	remaining: 10.8s
10478:	learn: 2680.2426129	total: 3m 37s	remaining: 10.8s
10479:	learn: 2680.2095449	total: 3m 37s	remaining: 10.8s
10480:	learn: 2679.3172411	total: 3m 37s	remaining: 10.8s
10481:	learn: 2679.2337263	total: 3m 37s	remaining: 10.7s
10482:	learn: 2678.9100568	total: 3m 37s	remaining: 10.7s
10483:	learn: 2678.7791454	total: 3m 37s	remaining: 10.7s
10484:	learn: 2678.7613069	total: 3m 37s	remaining: 10.7s
10485:	learn: 2678.7368272	total: 3m 37s	remaining: 10.7s
10486:	learn: 2678.1412940	total: 3m 37s	remaining: 10.6s
10487:	learn: 2677.3424563	total: 3m 37s	remaining: 10.6s
10488:	learn: 2676.8266328	total: 3m 37s	remaining: 10.6s
10489:	learn: 

10614:	learn: 2634.9656331	total: 3m 40s	remaining: 7.98s
10615:	learn: 2634.5274644	total: 3m 40s	remaining: 7.96s
10616:	learn: 2634.4512637	total: 3m 40s	remaining: 7.94s
10617:	learn: 2634.4149558	total: 3m 40s	remaining: 7.92s
10618:	learn: 2634.3873770	total: 3m 40s	remaining: 7.9s
10619:	learn: 2634.1878542	total: 3m 40s	remaining: 7.88s
10620:	learn: 2633.9183898	total: 3m 40s	remaining: 7.86s
10621:	learn: 2633.7504832	total: 3m 40s	remaining: 7.84s
10622:	learn: 2633.4800990	total: 3m 40s	remaining: 7.82s
10623:	learn: 2633.0130054	total: 3m 40s	remaining: 7.8s
10624:	learn: 2632.3440525	total: 3m 40s	remaining: 7.78s
10625:	learn: 2632.2635369	total: 3m 40s	remaining: 7.75s
10626:	learn: 2631.7982390	total: 3m 40s	remaining: 7.73s
10627:	learn: 2631.2560997	total: 3m 40s	remaining: 7.71s
10628:	learn: 2630.5476266	total: 3m 40s	remaining: 7.69s
10629:	learn: 2630.2236991	total: 3m 40s	remaining: 7.67s
10630:	learn: 2629.6396108	total: 3m 40s	remaining: 7.65s
10631:	learn: 26

10764:	learn: 2572.1937924	total: 3m 43s	remaining: 4.87s
10765:	learn: 2571.9945474	total: 3m 43s	remaining: 4.85s
10766:	learn: 2571.7659338	total: 3m 43s	remaining: 4.83s
10767:	learn: 2571.3237896	total: 3m 43s	remaining: 4.81s
10768:	learn: 2570.8352951	total: 3m 43s	remaining: 4.79s
10769:	learn: 2570.6785977	total: 3m 43s	remaining: 4.77s
10770:	learn: 2570.3622279	total: 3m 43s	remaining: 4.75s
10771:	learn: 2570.1497141	total: 3m 43s	remaining: 4.73s
10772:	learn: 2569.8826212	total: 3m 43s	remaining: 4.71s
10773:	learn: 2569.2321608	total: 3m 43s	remaining: 4.69s
10774:	learn: 2568.6404149	total: 3m 43s	remaining: 4.67s
10775:	learn: 2568.2784596	total: 3m 43s	remaining: 4.64s
10776:	learn: 2568.2592836	total: 3m 43s	remaining: 4.63s
10777:	learn: 2567.5274278	total: 3m 43s	remaining: 4.6s
10778:	learn: 2566.8591988	total: 3m 43s	remaining: 4.58s
10779:	learn: 2566.3817074	total: 3m 43s	remaining: 4.56s
10780:	learn: 2565.5541538	total: 3m 43s	remaining: 4.54s
10781:	learn: 2

10906:	learn: 2520.6053651	total: 3m 46s	remaining: 1.93s
10907:	learn: 2519.9854057	total: 3m 46s	remaining: 1.91s
10908:	learn: 2519.9251477	total: 3m 46s	remaining: 1.89s
10909:	learn: 2519.4594226	total: 3m 46s	remaining: 1.87s
10910:	learn: 2518.9194598	total: 3m 46s	remaining: 1.84s
10911:	learn: 2518.7285420	total: 3m 46s	remaining: 1.82s
10912:	learn: 2518.6999747	total: 3m 46s	remaining: 1.8s
10913:	learn: 2518.6369483	total: 3m 46s	remaining: 1.78s
10914:	learn: 2518.5912954	total: 3m 46s	remaining: 1.76s
10915:	learn: 2518.3285300	total: 3m 46s	remaining: 1.74s
10916:	learn: 2518.3017997	total: 3m 46s	remaining: 1.72s
10917:	learn: 2518.1072727	total: 3m 46s	remaining: 1.7s
10918:	learn: 2517.3193994	total: 3m 46s	remaining: 1.68s
10919:	learn: 2517.0644861	total: 3m 46s	remaining: 1.66s
10920:	learn: 2516.5442306	total: 3m 46s	remaining: 1.64s
10921:	learn: 2516.5308036	total: 3m 46s	remaining: 1.62s
10922:	learn: 2515.7670360	total: 3m 46s	remaining: 1.6s
10923:	learn: 251

In [47]:
print(model.best_score_)
fe = model.get_feature_importance(prettified=True)
fe.head(20)

{'learn': {'RMSE': 2488.14134077572}}


,Feature Id,Importances
0,FE1_OverallQual FE1_total_area_hq,22.065095
1,Neighborhood,4.225143
2,FE1_OverallCond FE1_total_area_hq,3.698398
3,FE1_OverallQual,3.400542
4,FE1_OverallQual^2,3.339200
5,FE0_HighQualSF,3.064243
6,FE1_FE1_total_area_hq^2,3.031507
7,FE1_FE1_total_area_hq,2.987441
8,LotArea,2.074109
9,KitchenQual,1.764908


In [48]:
sub_name = "data/submission_16.csv"
pd.DataFrame(model.predict(X_test), 
            index=range(1461, len(df)+1), 
            columns=['SalePrice']).reset_index().\
            rename(columns={'index': 'id'}).to_csv(sub_name, index=False)

# Score: 0.12177